# Spark Sql with hive support without installation of Hive. 
### Here we have to configure warehouse.dir and spark will create metastore_db folder to store all the table and database details which supported by apache derby

In [1]:
from os.path import abspath
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession

abs_path = abspath('spark-warehouse')
spark = SparkSession\
.builder\
.appName("MySparkAppWithHiveSupport")\
.config("spark.sql.warehouse.dir", abs_path)\
.enableHiveSupport()\
.getOrCreate()

sc = spark.sparkContext;
print(sc.version)
print(sc.getConf().getAll())

3.3.0
[('spark.sql.warehouse.dir', 'D:\\pythonWithPyspark\\Spark-practice\\spark-warehouse'), ('spark.driver.extraJavaOptions', '-XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED'), ('spark.app.name', 'MySparkAppWithHiveSupport'), ('spark.app.startTime', '1659548633454'), ('spark.executor.id', 'driver'), ('spark.driver.por

In [2]:
spark.sql("CREATE TABLE IF NOT EXISTS src (key INT, value STRING) USING hive;");

In [7]:
spark.sql("DESCRIBE TABLE src;")

DataFrame[col_name: string, data_type: string, comment: string]

In [9]:
spark.sql("INSERT INTO src (key, value) values (1, 'D1'),(2, 'D2'),(3, 'D3');");

In [12]:
DF1 = spark.sql("select * from src")
DF1.show()

+---+-----+
|key|value|
+---+-----+
|  1|   D1|
|  2|   D2|
|  3|   D3|
+---+-----+



In [26]:
#spark.sql("DROP DATABASE customer_data;")
spark.sql("create database if not exists customer_data COMMENT 'This is customer database';")
spark.sql("show databases;").show()
spark.sql("describe database extended customer_data;").show(10,True)

+-------------+
|    namespace|
+-------------+
|customer_data|
|      default|
+-------------+

+--------------+--------------------+
|     info_name|          info_value|
+--------------+--------------------+
|Namespace Name|       customer_data|
|       Comment|This is customer ...|
|      Location|file:/D:/pythonWi...|
|         Owner|      durgesh.pandey|
|    Properties|                    |
+--------------+--------------------+



In [29]:
spark.sql("use customer_data;")
path="sample_data/2015-12-12.csv"
csv_df=spark.read.options(inferSchema=True, header=True).csv(path)
csv_df.printSchema()
csv_df.show(10)
csv_df.write.saveAsTable("cust_data")

root
 |-- date: timestamp (nullable = true)
 |-- time: timestamp (nullable = true)
 |-- size: integer (nullable = true)
 |-- r_version: string (nullable = true)
 |-- r_arch: string (nullable = true)
 |-- r_os: string (nullable = true)
 |-- package: string (nullable = true)
 |-- version: string (nullable = true)
 |-- country: string (nullable = true)
 |-- ip_id: integer (nullable = true)

+-------------------+-------------------+-------+---------+------+-------+---------+-------+-------+-----+
|               date|               time|   size|r_version|r_arch|   r_os|  package|version|country|ip_id|
+-------------------+-------------------+-------+---------+------+-------+---------+-------+-------+-----+
|2015-12-12 00:00:00|2022-08-04 13:42:10| 257886|    3.2.2|  i386|mingw32| HistData|  0.7-6|     CZ|    1|
|2015-12-12 00:00:00|2022-08-04 13:24:37|1236751|    3.2.2|x86_64|mingw32|  RJSONIO|  1.3-0|     DE|    2|
|2015-12-12 00:00:00|2022-08-04 13:42:35|2077876|    3.2.2|  i386|mingw32|

In [30]:
spark.sql("SELECT * from cust_data where ip_id > 4").show(10)

+-------------------+-------------------+-------+---------+------+-------+---------+-------+-------+-----+
|               date|               time|   size|r_version|r_arch|   r_os|  package|version|country|ip_id|
+-------------------+-------------------+-------+---------+------+-------+---------+-------+-------+-----+
|2015-12-12 00:00:00|2022-08-04 02:58:25|  69772|    3.2.2|x86_64|mingw32|  acepack|1.3-3.3|     KR|  139|
|2015-12-12 00:00:00|2022-08-04 02:58:25| 266727|    3.2.2|x86_64|mingw32|gridExtra|  2.0.0|     KR|  139|
|2015-12-12 00:00:00|2022-08-04 02:58:26|1630065|    3.2.2|x86_64|mingw32|    Hmisc| 3.17-0|     KR|  139|
|2015-12-12 00:00:00|2022-08-04 02:58:32|  14621|       NA|    NA|     NA|megaptera|  1.0-0|     CN|11333|
|2015-12-12 00:00:00|2022-08-04 02:59:01|3195824|    3.2.2|x86_64|mingw32|     Rcpp| 0.12.2|     CN|11334|
|2015-12-12 00:00:00|2022-08-04 02:59:05|1114434|    3.2.2|x86_64|mingw32|     plyr|  1.8.3|     CN|11334|
|2015-12-12 00:00:00|2022-08-04 02:59